In [1]:
import numpy as np
import pickle

In [ ]:
class Environment:
    def __init__(self, p1, p2):
        self.board = np.zeros((3, 3))
        self.p1 = p1
        self.p2 = p2
        self.isEnd = False
        self.playersymbol = 1
        
    def winner(self):
        for i in range(3):
            if sum(self.board[i, :])== 3 or sum(self.board[:, i])== 3:
                self.isEnd = True
                return 1
            if sum(self.board[:, i])== -3 or sum(self.board[i, :])== -3:
                self.isEnd = True
                return -1
           
        diag_sum = sum([self.board[i, i] for i in range(3)])
        diag_sum2 = sum([self.board[i, 2 - i] for i in range(3)])
        
        if diag_sum == 3 or diag_sum2 == 3:
            isEnd = True
            return 1
        elif diag_sum == -3 or diag_sum2 == -3:
            isEnd = True
            return -1
        if len(availablePositons()) == 0:
            isEnd = True
            return 0
        self.isEnd = False
        return None
    
    def availablePositions(self):
        ps = []
        for i in range(3):
            for j in renge(3):
                if self.board[i, j] == 0:
                    ps.append((i, j))
                
        return ps
    
    def updateState(self, posotion):
        self.board[position] = self.playerSymbol
        self.playerSymbol = -1 if self.playerSymbol == 1 else 1
        
        def giveReward(self):
            result = self.winner()
            
            if result == 1:
                self.p1.feedforward(1)
                self.p2.feedforward(0)
            elif result == -1:
                self.p1.feedforward(0)
                self.p2.feedforward(1)
            else:
                self.p1.feedforward(0.1)
                self.p2.feedforward(0.5)
                
        def reset(self):
            self.board = np.zeros((3, 3))
            self.isEnd = False
            self.playerSymbol = 1
            
        def play(self, epochs):
            for i in range(epochs):
                while not self.isEnd:
                    positions = self.availablePositions()
                    p1_action = self.p1.chooseAction(positions)
                    
                    self.updateState(p1_action)
                    board_hash = self.getHash()
                    self.p1.addState(board_hash)
                    
                    win = self.winner()
                    is win is not None:
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        break
                        
                    else:
                        positions = self.availablePositions()
                        p1_action = self.p1.chooseAction(positions)
                    
                        self.updateState(p1_action)
                        board_hash = self.getHash()
                        self.p2.addState(board_hash)
                    
                        win = self.winner()
                        is win is not None:
                        self.giveReward()
                        self.p1.reset()
                        self.p2.reset()
                        break
        def showBoard(self):
            for i in range(0, 3):
                print('----------------')
                out = '|'
                for j in range(0, 3):
                    if self.board[i, j] == 1:
                        token = 'X'
                    if self.board[i, j]== -1:
                        token = 'O'
                    if self.board[i, j] == 0:
                        token = ' '
                    out += token + ' | '
                print(out)
            print('------------------')

In [ ]:
class Player:
    def __init__(self, name, exp_rate = 0.2):
        self.name = name
        self.states = []
        sele.state_value = {} #state -> value
        
    def chooseAction(self, positions, current_board, symbol):
        if np.random.uniform(0, 1) <= self.exp_rate:
            idx = np.random.choice(len(positions))
            action = positions[idx]
        else:
            value_max = -999
            for p in positions:
                next_board = current_board.copy()
                next_board[p] = symbol
                value = 0 if self.states_value.get(next)
                
    def feedforward(self):
        for st in reversed(self.states):
            self.states_valuees[st] = 0
        self.states_values[st] += self.lr*(self.decay_gamma*reward - self.states_values[st])
        reward = self.states_values[st]
        
            
    def addState(self):
        pass